In [1]:
import os
from dotenv import load_dotenv
from igdb.wrapper import IGDBWrapper
import json
import pandas as pd

load_dotenv()
ID = os.getenv('CLIENTID')
TOKEN = os.getenv('ACCESSTOKEN')

wrapper = IGDBWrapper(ID, TOKEN)

In [2]:
import time

keys = ['id', 'category', 'rating', 'rating_count', 'aggregated_rating', \
        'first_release_date', 'follows', 'game_modes', \
        'genres', 'involved_companies', 'keywords', 'summary', 'storyline', 'multiplayer_modes', \
        'name', 'platforms', 'player_perspectives', 'similar_games', 'themes']

batch = {k: [] for k in keys}

byte_array = wrapper.api_request(
            'games',
            'fields id, name, rating, aggregated_rating, category, storyline, \
             first_release_date, follows, rating_count, game_modes, genres, involved_companies,\
             keywords, multiplayer_modes, platforms, summary, player_perspectives,\
             similar_games, themes, expanded_games; limit 500; \
             where category = 0 & first_release_date > 1; sort first_release_date;'
          )

count = 1

modfields = ['game_modes', 'genres', 'platforms', 'involved_companies', 'keywords',\
              'multiplayer_modes', 'player_perspectives', 'similar_games', 'themes']

while len(byte_array) > 2:
        
    for i in json.loads(byte_array.decode('utf-8')):
                              
        for modfield in modfields:
            try:
                i[modfield] = ' '.join([str(s) for s in i[modfield]])
            except KeyError:
                i[modfield] = ''
    
        for key in keys:
            try:
                batch[key].append(i[key])
            except KeyError:
                batch[key].append(None)
            
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'games',
            f'fields id, name, rating, aggregated_rating, category, storyline,\
             first_release_date, follows, rating_count, summary, game_modes, genres, involved_companies,\
             keywords, multiplayer_modes, platforms, player_perspectives,\
             similar_games, themes, expanded_games; limit 500; offset {500*count}; \
             where category = 0 & first_release_date > 1; sort first_release_date;'
          )
    count += 1


base = pd.DataFrame(batch)

In [3]:
base.set_index(['id'], inplace=True)

In [4]:
byte_array = wrapper.api_request(
            'keywords',
            f'fields name; limit 500;'
          )

count = 1

keyworddict = {'id': [], 'name':[]}

while len(byte_array) > 2:
        
    for i in json.loads(byte_array.decode('utf-8')):
        keyworddict['id'].append(i['id'])
        keyworddict['name'].append(i['name'])
    
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'keywords',
            f'fields name; limit 500; offset {500*count};'
          )
    
    count += 1
        
keywords = pd.DataFrame(keyworddict)


In [5]:
keywords.set_index(['id'],inplace=True)

In [6]:
byte_array = wrapper.api_request(
            'multiplayer_modes',
            'fields *;exclude checksum, game; limit 500;'
          )

count = 1

keys =  ['id', 'campaigncoop', 'dropin', 'lancoop', 'offlinecoop', \
         'offlinecoopmax', 'offlinemax', 'onlinecoop', 'onlinecoopmax', \
         'onlinemax', 'platform', 'splitscreen']

multiplayermodesdict = {k: [] for k in keys}

while len(byte_array) > 2:
    
    for i in json.loads(byte_array.decode('utf-8')):
        for key in keys:
            try:
                multiplayermodesdict[key].append(i[key])
            except KeyError:
                multiplayermodesdict[key].append(None)
    
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'multiplayer_modes',
            f'fields *;exclude checksum, game; limit 500; offset {500*count};'
          )
    
    count += 1
        
multimodes = pd.DataFrame(multiplayermodesdict)

In [7]:
multimodes.set_index(['id'], inplace=True)

In [8]:
def keywordsidtoname(ids: str):
    if len(ids) > 0:
        try:
            names = ' '.join(keywords.loc[[int(i) for i in ids.split()], 'name'].values)
        except KeyError:
            names = ''
    else:
        names = ''
    return names

base['keywords'] = base['keywords'].apply(keywordsidtoname)

In [9]:
def modeidtotp(ids: str):
    blankmode =(('campaigncoop', False), ('dropin', False), ('lancoop', False), ('offlinecoop', False),
                ('offlinecoopmax', 0.0), ('offlinemax', 0.0), ('onlinecoop', False), ('onlinecoopmax', 0.0),
                ('onlinemax', 0.0), ('platform', 0.0), ('splitscreen', False))
    if ids != '':
        try:
            modes = tuple(multimodes.loc[int(ids.split()[0])].items())
        except KeyError:
            modes = blankmode
    else:
        modes = blankmode
    return modes

for i in base.index:
    mode = modeidtotp(base.loc[i, 'multiplayer_modes'])
    for col, val in mode:
        base.loc[i, col] = val

In [10]:
import os.path

datasetdir = os.path.join('.', 'dataset')
filename = 'games.csv'
path = os.path.join(datasetdir, filename)
base.to_csv(path)

In [11]:
byte_array = wrapper.api_request(
            'platforms',
            f'fields name; limit 500;'
          )



for n, i in enumerate(json.loads(byte_array.decode('utf-8'))):
    print(n, i)

0 {'id': 158, 'name': 'Commodore CDTV'}
1 {'id': 339, 'name': 'Sega Pico'}
2 {'id': 8, 'name': 'PlayStation 2'}
3 {'id': 39, 'name': 'iOS'}
4 {'id': 94, 'name': 'Commodore Plus/4'}
5 {'id': 144, 'name': 'AY-3-8710'}
6 {'id': 88, 'name': 'Odyssey'}
7 {'id': 90, 'name': 'Commodore PET'}
8 {'id': 237, 'name': 'Sol-20'}
9 {'id': 6, 'name': 'PC (Microsoft Windows)'}
10 {'id': 44, 'name': 'Tapwave Zodiac'}
11 {'id': 68, 'name': 'ColecoVision'}
12 {'id': 165, 'name': 'PlayStation VR'}
13 {'id': 129, 'name': 'Texas Instruments TI-99'}
14 {'id': 133, 'name': 'Philips Videopac G7000'}
15 {'id': 134, 'name': 'Acorn Electron'}
16 {'id': 378, 'name': 'Gamate'}
17 {'id': 135, 'name': 'Hyper Neo Geo 64'}
18 {'id': 156, 'name': 'Thomson MO5'}
19 {'id': 163, 'name': 'SteamVR'}
20 {'id': 142, 'name': 'PC-50X Family'}
21 {'id': 148, 'name': 'AY-3-8607'}
22 {'id': 146, 'name': 'AY-3-8605'}
23 {'id': 147, 'name': 'AY-3-8606'}
24 {'id': 149, 'name': 'PC-98'}
25 {'id': 25, 'name': 'Amstrad CPC'}
26 {'id': 38

In [18]:
byte_array = wrapper.api_request(
            'genres',
            f'fields name; limit 500;'
          )



for n, i in enumerate(json.loads(byte_array.decode('utf-8'))):
    print(n, i)

0 {'id': 4, 'name': 'Fighting'}
1 {'id': 5, 'name': 'Shooter'}
2 {'id': 7, 'name': 'Music'}
3 {'id': 8, 'name': 'Platform'}
4 {'id': 9, 'name': 'Puzzle'}
5 {'id': 10, 'name': 'Racing'}
6 {'id': 11, 'name': 'Real Time Strategy (RTS)'}
7 {'id': 12, 'name': 'Role-playing (RPG)'}
8 {'id': 13, 'name': 'Simulator'}
9 {'id': 14, 'name': 'Sport'}
10 {'id': 15, 'name': 'Strategy'}
11 {'id': 16, 'name': 'Turn-based strategy (TBS)'}
12 {'id': 24, 'name': 'Tactical'}
13 {'id': 26, 'name': 'Quiz/Trivia'}
14 {'id': 25, 'name': "Hack and slash/Beat 'em up"}
15 {'id': 30, 'name': 'Pinball'}
16 {'id': 31, 'name': 'Adventure'}
17 {'id': 33, 'name': 'Arcade'}
18 {'id': 34, 'name': 'Visual Novel'}
19 {'id': 32, 'name': 'Indie'}
20 {'id': 35, 'name': 'Card & Board Game'}
21 {'id': 36, 'name': 'MOBA'}
22 {'id': 2, 'name': 'Point-and-click'}


In [13]:
byte_array = wrapper.api_request(
            'player_perspectives',
            f'fields name; limit 500;'
          )



for n, i in enumerate(json.loads(byte_array.decode('utf-8'))):
    print(n, i)

0 {'id': 1, 'name': 'First person'}
1 {'id': 2, 'name': 'Third person'}
2 {'id': 3, 'name': 'Bird view / Isometric'}
3 {'id': 5, 'name': 'Text'}
4 {'id': 4, 'name': 'Side view'}
5 {'id': 7, 'name': 'Virtual Reality'}
6 {'id': 6, 'name': 'Auditory'}


In [19]:
byte_array = wrapper.api_request(
            'themes',
            f'fields name; limit 500;'
          )



for n, i in enumerate(json.loads(byte_array.decode('utf-8'))):
    print(n, i)

0 {'id': 20, 'name': 'Thriller'}
1 {'id': 18, 'name': 'Science fiction'}
2 {'id': 1, 'name': 'Action'}
3 {'id': 19, 'name': 'Horror'}
4 {'id': 21, 'name': 'Survival'}
5 {'id': 17, 'name': 'Fantasy'}
6 {'id': 22, 'name': 'Historical'}
7 {'id': 23, 'name': 'Stealth'}
8 {'id': 27, 'name': 'Comedy'}
9 {'id': 28, 'name': 'Business'}
10 {'id': 31, 'name': 'Drama'}
11 {'id': 32, 'name': 'Non-fiction'}
12 {'id': 35, 'name': 'Kids'}
13 {'id': 33, 'name': 'Sandbox'}
14 {'id': 38, 'name': 'Open world'}
15 {'id': 39, 'name': 'Warfare'}
16 {'id': 41, 'name': '4X (explore, expand, exploit, and exterminate)'}
17 {'id': 34, 'name': 'Educational'}
18 {'id': 43, 'name': 'Mystery'}
19 {'id': 40, 'name': 'Party'}
20 {'id': 44, 'name': 'Romance'}
21 {'id': 42, 'name': 'Erotic'}
